In [15]:
"""
Script d'augmentation de données pour des synopsis de films d'horreur (en français)

Ce script permet d'enrichir un jeu de données textuelles (des synopsis de films) en créant 
une version augmentée de chaque synopsis, dans le but d'améliorer la robustesse de modèles 
de traitement du langage naturel (NLP) utilisés dans des tâches comme la classification ou 
la génération de texte.

Fonctionnalités principales :
-----------------------------
1. Lecture des fichiers texte depuis un dossier source (INPUT_DIR).
2. Tokenisation des textes en mots (en français).
3. Substitution aléatoire de certains mots par des synonymes :
   - Soit issus d'un dictionnaire personnalisé orienté "horreur" (thèmes, créatures, lieux...),
   - Soit extraits automatiquement depuis WordNet (en français) via NLTK.
4. Écriture d'une version modifiée de chaque synopsis dans un nouveau fichier `.txt`, 
   dans un dossier de sortie (OUTPUT_DIR).

Avantages :
-----------
- Permet de diversifier les données sans changer radicalement le sens des phrases.
- Génère automatiquement 1 nouveau fichier par synopsis.
- S'intègre facilement dans un pipeline de prétraitement pour les tâches NLP.

Pré-requis :
------------
- Python 3
- nltk (`pip install nltk`)
- Fichiers `.txt` dans le dossier `../film_synopses` (un synopsis par fichier)

Résultat :
----------
- Le dossier `synopsis_augmente` contiendra une version enrichie de chaque synopsis original,
  avec 1 ou 2 substitutions de mots par texte.
"""

import os
import random
import shutil
import nltk
from nltk.corpus import wordnet as wn
from nltk.tokenize import word_tokenize

# Téléchargements nécessaires
nltk.download('punkt')
nltk.download('omw-1.4')
nltk.download('wordnet')

# Dossiers
INPUT_DIR = "../film_synopses"
OUTPUT_DIR = "synopsis_augmente"

# Vocabulaire horreur personnalisé
horreur = {
    # Thèmes surnaturels
    "esprit": ["âme errante", "entité", "présence invisible"],
    "fantôme": ["spectre", "apparition", "revenant"],
    "démon": ["entité maléfique", "créature infernale", "diable"],
    "possédé": ["habité", "contrôlé", "sous emprise"],
    "malédiction": ["sort", "condamnation", "fléau mystique"],
    "sorcellerie": ["magie noire", "sabbat", "envoûtement"],
    "rituel": ["cérémonie occulte", "invocation", "rite ancien"],
    "exorcisme": ["purification", "chasse aux démons", "bannissement"],
    "enfer": ["abîme", "royaume infernal", "géhenne"],
    "pacte": ["accord maudit", "serment démoniaque", "marché noir"],
    "apocalypse": ["fin du monde", "jugement dernier", "cataclysme"],

    # Violence et gore
    "tueur": ["assassin", "psychopathe", "meurtrier"],
    "meurtre": ["homicide", "crime sanglant", "assassinat"],
    "massacre": ["boucherie", "tuerie", "hécatombe"],
    "sacrifice": ["offrande sanglante", "immolation", "rite de sang"],
    "torture": ["supplice", "souffrance", "agonie lente"],
    "sang": ["traînée sanglante", "fluide vital", "giclée rouge"],
    "cadavre": ["corps sans vie", "dépouille", "restes humains"],
    "hurlement": ["cri strident", "appel à l'aide", "sanglot déchirant"],
    "agonie": ["souffrance finale", "derniers instants", "lente douleur"],
    "déchiqueté": ["démembré", "éventré", "lacéré"],
    "mutilation": ["amputation", "défiguration", "blessure extrême"],

    # Peur, ambiance
    "peur": ["effroi", "terreur", "frayeur"],
    "terreur": ["épouvante", "angoisse", "panique"],
    "cauchemar": ["rêve macabre", "hallucination nocturne", "vision d'horreur"],
    "angoisse": ["stress profond", "peur viscérale", "malaise constant"],
    "panique": ["détresse", "affolement", "crise de peur"],
    "crie": ["hurle", "braille", "gémit"],
    "épouvante": ["horreur", "effroi pur", "terreur froide"],
    "frisson": ["chair de poule", "tremblement", "sensation glaciale"],
    "folie": ["délire", "aliénation", "perte de raison"],
    "paranoïa": ["méfiance extrême", "délire de persécution", "obsession"],
    "hallucination": ["vision irréelle", "illusion", "dérive mentale"],

    # Lieux et objets
    "maison": ["manoir", "demeure sinistre", "bâtisse hantée"],
    "manoir": ["château lugubre", "villa hantée", "vieille bâtisse"],
    "hôpital": ["asile", "centre abandonné", "clinique vide"],
    "asile": ["hôpital psychiatrique", "institution fermée", "prison mentale"],
    "forêt": ["bois obscur", "sous-bois hanté", "jungle maudite"],
    "cimetière": ["nécropole", "lieu de repos éternel", "terre des morts"],
    "grenier": ["pièce oubliée", "recoin poussiéreux", "étage noir"],
    "cave": ["sous-sol", "cellier lugubre", "souterrain"],
    "miroir": ["reflet maudit", "verre hanté", "surface trompeuse"],
    "poupée": ["jouet maléfique", "figurine animée", "objet possédé"],
    "croix": ["symbole sacré", "croix inversée", "relique"],
    "talisman": ["amulette", "objet mystique", "artefact sacré"],

    # Créatures
    "monstre": ["créature", "abomination", "bête difforme"],
    "zombie": ["mort-vivant", "cadavre animé", "créature décomposée"],
    "vampire": ["buveur de sang", "sanguinaire", "immortel nocturne"],
    "loup-garou": ["bête lunaire", "homme-loup", "créature lycanthrope"],
    "momie": ["corps embaumé", "ancien roi maudit", "être bandé"],
    "clown": ["bouffon maléfique", "farceur tordu", "joker démoniaque"],
    "chasseur": ["traqueur", "pisteur", "tueur silencieux"],
    "entité": ["forme invisible", "créature inconnue", "présence paranormale"],
    "ombre": ["silhouette noire", "absence de lumière", "forme mouvante"],
    "spectre": ["fantôme", "esprit", "vision éthérée"],
    "revenant": ["âme en peine", "fantôme du passé", "mort revenu"],

    # Innocence corrompue
    "enfant": ["âme innocente", "orphelin hanté", "petit être fragile"],
    "bébé": ["nouveau-né", "poupon", "petit être"],
    "innocent": ["pur", "candide", "naïf"],
    "orphelin": ["sans famille", "enfant abandonné", "isolé"],
    "famille": ["proches", "parents", "lignée"],
    "adoption": ["accueil", "prise en charge", "nouvelle famille"]

}

# Fonction pour récupérer des synonymes d’un mot
def get_synonyms(word):
    # Cherche d'abord dans un dictionnaire personnalisé d'horreur
    synonyms = set(horreur.get(word.lower(), []))
    
    # Ensuite, ajoute les synonymes trouvés via WordNet en français
    for synset in wn.synsets(word, lang='fra'):
        for lemma in synset.lemmas('fra'):
            syn = lemma.name().replace("_", " ").lower()
            # On évite les doublons et les synonymes trop longs
            if syn != word.lower() and len(syn) < 20:
                synonyms.add(syn)
    
    return list(synonyms)

# Fonction principale d’augmentation : remplace aléatoirement jusqu’à `n` mots par des synonymes
def augment(text, n=2):
    # Découpe le texte en mots
    words = word_tokenize(text, language='french')
    
    # Sélectionne les mots pour lesquels des synonymes sont disponibles
    candidates = [w for w in words if get_synonyms(w)]
    random.shuffle(candidates)  # Mélange pour introduire de la variété
    
    # Remplace jusqu’à `n` mots par des synonymes
    for i in range(min(n, len(candidates))):
        w = candidates[i]
        syns = get_synonyms(w)
        if syns:
            substitute = random.choice(syns)
            # Substitution dans la liste des mots
            words = [substitute if word == w else word for word in words]
    
    # Retourne le texte reconstruit
    return " ".join(words)

# Fonction principale d'exécution du script
def main():
    # Nettoie le dossier de sortie s’il existe déjà
    if os.path.exists(OUTPUT_DIR):
        shutil.rmtree(OUTPUT_DIR)
    os.makedirs(OUTPUT_DIR)

    # Parcourt tous les fichiers texte du dossier d’entrée
    for filename in os.listdir(INPUT_DIR):
        if filename.endswith(".txt"):
            with open(os.path.join(INPUT_DIR, filename), "r", encoding="utf-8") as f:
                original = f.read()

            # Crée une version augmentée du texte
            augmented = augment(original)

            # Prépare le chemin du fichier de sortie
            base = os.path.splitext(filename)[0]
            output_path = os.path.join(OUTPUT_DIR, f"{base}_augmente.txt")

            # Sauvegarde le texte modifié
            with open(output_path, "w", encoding="utf-8") as f:
                f.write(augmented)

    print("Augmentation terminée : 1 fichier généré par synopsis.")

# Lancement du script si exécuté directement
if __name__ == "__main__":
    main()



[nltk_data] Downloading package punkt to /home/ines/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/ines/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to /home/ines/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Augmentation terminée : 1 fichier généré par synopsis.
